In [136]:
import requests
import json
from alpha_vantage.foreignexchange import ForeignExchange
import numpy as np
import keys
import csv
import uuid

In [137]:
def get_gold_price(api_key):
    fx = ForeignExchange(key=api_key)
    result, _ = fx.get_currency_exchange_rate(from_currency='XAU', to_currency='USD')
    return result['9. Ask Price']

In [138]:
api_key = keys.ALPHA_VANTAGE
result = get_gold_price(api_key)
print(json.dumps(result, indent=4))

"1960.13000000"


In [139]:
#Add a file keys.py in the folder, and input your own API KEYS

class WiseClient:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://api.sandbox.transferwise.tech/"
        self.headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        self.gold = float(get_gold_price(api_key))
    def generate_uuid(self):
        url = "https://www.uuidgenerator.net/api/guid"

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        return response.text

    def get_borderless_ID(self,profile_id):
        url = self.base_url + f'/v1/borderless-accounts?profileId={profile_id}'
        response = requests.get(url, headers=self.headers)

        return response.json()

        
    def get_balance(self, profile_id):
        url = self.base_url + f'v1/borderless-accounts?profileId={profile_id}'
        response = requests.get(url, headers=self.headers)
        balance_data = response.json()
        return balance_data  # This should return your account balances in different currencies

    def get_rates(self,source,target):
        url = self.base_url + f'/v1/rates?source={source}&target={target}'
        response = requests.get(url,headers=self.headers)
        return response.json()
    
    def get_true_change_target(self,source,target,targetAmount):
        url = self.base_url + f'/v1/quotes?source={source}&target={target}&rateType=FIXED&targetAmount={targetAmount}'
        response = requests.get(url,headers=self.headers)
        return response.json()
    
    def get_true_change_source(self,source,target,sourceAmount):
        url = self.base_url + f'/v1/quotes?source={source}&target={target}&rateType=FIXED&sourceAmount={sourceAmount}'
        response = requests.get(url,headers=self.headers)
        return response.json()
    def create_quote(self,profile_id, source_currency, target_currency, source_amount, target_account):
        url = self.base_url +f"/v3/profiles/{profile_id}/quotes"
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json',
        }
        payload = json.dumps({
            'sourceCurrency': source_currency,
            'targetCurrency': target_currency,
            'sourceAmount': source_amount,
            'targetAmount': None,
            'payOut': 'BALANCE',
            'preferredPayIn': None,
            'targetAccount': target_account,
            'paymentMetadata': {
                'transferNature': 'MOVING_MONEY_BETWEEN_OWN_ACCOUNTS'
            }
        })
        response = requests.post(url, headers=headers, data=payload)
        quote_data = response.json()
        return quote_data

    def update_quote(self,profile_id, quote_id, target_account):
        url = f"https://api.sandbox.transferwise.tech/v3/profiles/{profile_id}/quotes/{quote_id}"
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/merge-patch+json',
        }
        payload = json.dumps({
            'targetAccount': target_account,
            'payOut': 'BALANCE',
            'paymentMetadata': {
                'transferNature': 'MOVING_MONEY_BETWEEN_OWN_ACCOUNTS'
            }
        })
        response = requests.patch(url, headers=headers, data=payload)
        return response.json()

    def initiate_balance_conversion(self,profile_id, quote_id):
        url = self.base_url + f"/v2/profiles/{profile_id}/balance-movements"
        # url = f'https://sandbox.transferwise.tech/flows/balances/{profile_id}/convert' #158818
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json',
            'X-idempotence-uuid': str(uuid.uuid4()),  # generate a new UUID for each request
        }
        payload = json.dumps({
            'quoteId': quote_id
        })
        response = requests.post(url, headers=headers, data=payload)
        return response.json()

    def check_id(self,account_id):
        url = self.base_url+ '/v1/accounts/' + account_id
        response = requests.get(url, headers=self.headers)
        return response.json()
    
    def TAL_weights(self,source):
        weights = np.array([
            self.get_rates('CHF', source)[0]['rate'] * 100,
            self.get_rates('EUR', source)[0]['rate']  * 250,
            self.get_rates('GBP', source)[0]['rate']  * 50,
            self.get_rates('JPY', source)[0]['rate']  * 18000,
            self.get_rates('CNY', source)[0]['rate']  * 1600,
            self.get_rates('SGD', source)[0]['rate']  * 80,
            self.get_rates('USD',source)[0]['rate']  * self.gold*0.2
        ])

        response = json.loads(self.TAL_rate(source))
        
        weights /= response['total_amount']

        return weights
    
    def TAL_rate(self,source):
        chf = self.get_true_change_target(source,'CHF',100)
        chf_amount = chf['sourceAmount']
        chf_fee = chf['fee']
        eur = self.get_true_change_target(source,'EUR',250)
        eur_amount = eur['sourceAmount']
        eur_fee = eur['fee']
        gbp = self.get_true_change_target(source,'GBP',50)
        gbp_amount = gbp['sourceAmount']
        gbp_fee = gbp['fee']
        jpy = self.get_true_change_target(source,'JPY',18000)
        jpy_amount = jpy['sourceAmount']
        jpy_fee = jpy['fee']
        cny = self.get_true_change_target(source,'CNY',1600)
        cny_amount = cny['sourceAmount']
        cny_fee = cny['fee']
        sgd = self.get_true_change_target(source,'SGD',80)
        sgd_amount = sgd['sourceAmount']
        sgd_fee = sgd['fee']
        gold = self.gold*0.2
        usdgold = self.get_true_change_target(source,'USD',gold)
        usdgold_amount = usdgold['sourceAmount']
        usdgold_fee = usdgold['fee']

        result_dict = {
            'total_amount': chf_amount + eur_amount + gbp_amount + jpy_amount + cny_amount + sgd_amount + usdgold_amount,
            'total_fee': chf_fee + eur_fee + gbp_fee + jpy_fee + cny_fee + sgd_fee + usdgold_fee,
            'chf': {'amount': chf_amount, 'fee': chf_fee},
            'eur': {'amount': eur_amount, 'fee': eur_fee},
            'gbp': {'amount': gbp_amount, 'fee': gbp_fee},
            'jpy': {'amount': jpy_amount, 'fee': jpy_fee},
            'cny': {'amount': cny_amount, 'fee': cny_fee},
            'sgd': {'amount': sgd_amount, 'fee': sgd_fee},
            'usdgold': {'amount': usdgold_amount, 'fee': usdgold_fee}
        }
        
        # Convert the dictionary to a JSON string
        result_json = json.dumps(result_dict,indent=4)
        
        return result_json

    def True_TAL_Change(self,source,sourceAmount):
        weights = self.TAL_weights(source)

        distribution = sourceAmount*weights

        chf = self.get_true_change_source(source,'CHF',distribution[0])
        chf_amount = chf['targetAmount']
        chf_fee = chf['fee']
        eur = self.get_true_change_source(source,'EUR',distribution[1])
        eur_amount = eur['targetAmount']
        eur_fee = eur['fee']
        gbp = self.get_true_change_source(source,'GBP',distribution[2])
        gbp_amount = gbp['targetAmount']
        gbp_fee = gbp['fee']
        jpy = self.get_true_change_source(source,'JPY',distribution[3])
        jpy_amount = jpy['targetAmount']
        jpy_fee = jpy['fee']
        cny = self.get_true_change_source(source,'CNY',distribution[4])
        cny_amount = cny['targetAmount']
        cny_fee = cny['fee']
        sgd = self.get_true_change_source(source,'SGD',distribution[5])
        sgd_amount = sgd['targetAmount']
        sgd_fee = sgd['fee']
        usdgold = self.get_true_change_source(source,'USD',distribution[6])
        gold_amount = usdgold['targetAmount'] / self.gold
        gold_fee = usdgold['fee']

        result_dict = {
            'chf': {'amount': chf_amount, 'fee': chf_fee},
            'eur': {'amount': eur_amount, 'fee': eur_fee},
            'gbp': {'amount': gbp_amount, 'fee': gbp_fee},
            'jpy': {'amount': jpy_amount, 'fee': jpy_fee},
            'cny': {'amount': cny_amount, 'fee': cny_fee},
            'sgd': {'amount': sgd_amount, 'fee': sgd_fee},
            'gold': {'amount': gold_amount, 'fee': gold_fee},
            'TAL':{'amount': chf_amount*10,f'total_fee in {source}': chf_fee + eur_fee + gbp_fee + jpy_fee + cny_fee + sgd_fee + gold_fee}
        }
        result_json = json.dumps(result_dict,indent=4)
        
        return result_json

    
wise_client = WiseClient(keys.API)

In [140]:
distribution = wise_client.True_TAL_Change('CHF',20000)
print(distribution)

{
    "chf": {
        "amount": 1800.09,
        "fee": 0.92
    },
    "eur": {
        "amount": 4483.01,
        "fee": 19.14
    },
    "gbp": {
        "amount": 896.26,
        "fee": 4.93
    },
    "jpy": {
        "amount": 321838,
        "fee": 15.06
    },
    "cny": {
        "amount": 28483.08,
        "fee": 40.89
    },
    "sgd": {
        "amount": 1433.76,
        "fee": 4.95
    },
    "gold": {
        "amount": 3.586297847591741,
        "fee": 27.47
    },
    "TAL": {
        "amount": 18000.899999999998,
        "total_fee in CHF": 113.36
    }
}


In [141]:
weights = wise_client.TAL_weights('BRL')
print(weights)

[0.08840466 0.21307156 0.04986825 0.09972567 0.17133371 0.04614864
 0.30106474]


In [142]:
tal_rate = json.loads(wise_client.TAL_rate('BRL'))
print(tal_rate['total_amount'])

6239.23


In [143]:
# https://github.com/transferwise/api-docs/blob/master/source/includes/_payouts.md

In [144]:
profile_id = keys.PROFILE_ID
balances = wise_client.get_balance(profile_id)

# Find USD and EUR balance ids
usd_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'USD'), None)
eur_balance_id = next((balance['id'] for balance in balances[0]['balances'] if balance['currency'] == 'EUR'), None)

print(usd_balance_id,eur_balance_id)
print(json.dumps(balances,indent=4))


158818 158817
[
    {
        "id": 64440,
        "profileId": 16841712,
        "recipientId": 148941324,
        "creationTime": "2023-07-11T12:27:53.899Z",
        "modificationTime": "2023-07-11T12:27:53.899Z",
        "active": true,
        "eligible": true,
        "balances": [
            {
                "id": 158818,
                "balanceType": "AVAILABLE",
                "currency": "USD",
                "amount": {
                    "value": 2991430.24,
                    "currency": "USD"
                },
                "reservedAmount": {
                    "value": 0.0,
                    "currency": "USD"
                },
                "bankDetails": null
            },
            {
                "id": 158819,
                "balanceType": "AVAILABLE",
                "currency": "AUD",
                "amount": {
                    "value": 4001496.24,
                    "currency": "AUD"
                },
                "reservedAmount": {


In [145]:
rates = wise_client.get_rates('USD','EUR')
print(json.dumps(rates,indent=4))

[
    {
        "rate": 0.90107,
        "source": "USD",
        "target": "EUR",
        "time": "2023-07-12T14:11:45+0000"
    }
]


In [146]:
true_change = wise_client.get_true_change_target('USD','EUR',1000)

true_change

{'source': 'USD',
 'target': 'EUR',
 'sourceAmount': 1120.72,
 'targetAmount': 1000.0,
 'type': 'REGULAR',
 'rate': 0.901125,
 'createdTime': '2023-07-12T14:12:15.978Z',
 'rateType': 'FIXED',
 'deliveryEstimate': '2023-07-13T09:31:00Z',
 'fee': 11.0,
 'feeDetails': {'transferwise': 5.66,
  'payIn': 5.34,
  'discount': 0,
  'priceSetId': 129,
  'partner': 0.0},
 'allowedProfileTypes': ['PERSONAL', 'BUSINESS'],
 'guaranteedTargetAmount': False,
 'ofSourceAmount': False}

In [147]:
true_change = wise_client.get_true_change_source('USD','EUR',1000)
true_change

{'source': 'USD',
 'target': 'EUR',
 'sourceAmount': 1000.0,
 'targetAmount': 891.68,
 'type': 'REGULAR',
 'rate': 0.901125,
 'createdTime': '2023-07-12T14:12:16.225Z',
 'rateType': 'FIXED',
 'deliveryEstimate': '2023-07-13T09:31:00Z',
 'fee': 10.48,
 'feeDetails': {'transferwise': 5.14,
  'payIn': 5.34,
  'discount': 0,
  'priceSetId': 129,
  'partner': 0.0},
 'allowedProfileTypes': ['PERSONAL', 'BUSINESS'],
 'guaranteedTargetAmount': False,
 'ofSourceAmount': True}

In [148]:
profile_id = keys.PROFILE_ID # replace this with your profile id


In [149]:
wise_client.get_balance(profile_id)

[{'id': 64440,
  'profileId': 16841712,
  'recipientId': 148941324,
  'creationTime': '2023-07-11T12:27:53.899Z',
  'modificationTime': '2023-07-11T12:27:53.899Z',
  'active': True,
  'eligible': True,
  'balances': [{'id': 158818,
    'balanceType': 'AVAILABLE',
    'currency': 'USD',
    'amount': {'value': 2991430.24, 'currency': 'USD'},
    'reservedAmount': {'value': 0.0, 'currency': 'USD'},
    'bankDetails': None},
   {'id': 158819,
    'balanceType': 'AVAILABLE',
    'currency': 'AUD',
    'amount': {'value': 4001496.24, 'currency': 'AUD'},
    'reservedAmount': {'value': 0.0, 'currency': 'AUD'},
    'bankDetails': None},
   {'id': 158817,
    'balanceType': 'AVAILABLE',
    'currency': 'EUR',
    'amount': {'value': 2005897.71, 'currency': 'EUR'},
    'reservedAmount': {'value': 0.0, 'currency': 'EUR'},
    'bankDetails': None},
   {'id': 158816,
    'balanceType': 'AVAILABLE',
    'currency': 'GBP',
    'amount': {'value': 999984.0, 'currency': 'GBP'},
    'reservedAmount': {

In [150]:
# Create a quote
quote_data = wise_client.create_quote(profile_id,'USD','EUR',1000,148941324)

print(json.dumps(quote_data,indent=4))


{
    "sourceAmount": 1000.0,
    "guaranteedTargetAmountAllowed": false,
    "targetAmountAllowed": true,
    "paymentOptions": [
        {
            "formattedEstimatedDelivery": "in 2 hours",
            "estimatedDeliveryDelays": [],
            "allowedProfileTypes": [
                "PERSONAL",
                "BUSINESS"
            ],
            "feePercentage": 0.0074,
            "estimatedDelivery": "2023-07-12T16:00:00Z",
            "disabled": false,
            "sourceAmount": 1000.0,
            "targetAmount": 894.49,
            "sourceCurrency": "USD",
            "targetCurrency": "EUR",
            "payOut": "BALANCE",
            "fee": {
                "transferwise": 4.27,
                "payIn": 3.09,
                "discount": 0,
                "total": 7.36,
                "priceSetId": 130,
                "partner": 0.0
            },
            "price": {
                "priceSetId": 130,
                "total": {
                    "type": "TO

In [151]:
new_quote_data = wise_client.update_quote(profile_id,quote_data['id'],148941324)
print(json.dumps(new_quote_data,indent=4))

{
    "sourceAmount": 1000.0,
    "guaranteedTargetAmountAllowed": false,
    "targetAmountAllowed": true,
    "paymentOptions": [
        {
            "formattedEstimatedDelivery": "in 2 hours",
            "estimatedDeliveryDelays": [],
            "allowedProfileTypes": [
                "PERSONAL",
                "BUSINESS"
            ],
            "feePercentage": 0.0074,
            "estimatedDelivery": "2023-07-12T16:00:00Z",
            "disabled": false,
            "sourceAmount": 1000.0,
            "targetAmount": 894.49,
            "sourceCurrency": "USD",
            "targetCurrency": "EUR",
            "payOut": "BALANCE",
            "fee": {
                "transferwise": 4.27,
                "payIn": 3.09,
                "discount": 0,
                "total": 7.36,
                "priceSetId": 130,
                "partner": 0.0
            },
            "price": {
                "priceSetId": 130,
                "total": {
                    "type": "TO

In [152]:
conversion_data = wise_client.initiate_balance_conversion(profile_id,new_quote_data['id'])

print(json.dumps(conversion_data,indent=4))

{
    "code": "quote.payment-option-not-found",
    "message": "Paying with your balance is not valid for this transaction."
}
